In [1]:
from random import random
from queue import Queue
from prevo.misc import PeriodicThreadedSystem
from prevo.plot import OscilloGraph, NumericalGraph
import oclock
%matplotlib qt5

# Dummy sensors

In [2]:
class ManipBase(PeriodicThreadedSystem):
    
    def __init__(self, interval=0.1):
        super().__init__(interval=interval, precise=False)
        self.queue = Queue()
        
    def get_values(self):
        """Define in subclasses"""
        pass
        
    def _update(self):
        with oclock.measure_time() as data:
            values = self._get_values()
        data['values'] = values
        data['name'] = self.name
        self.queue.put(data)

In [3]:
class Manip1(ManipBase):
    
    name = 'A'
    data_types = ('P (Pa)', 'T (°C)', 'P (Pa)')
    
    def _get_values(self):
        val1 = random()
        val2 = 25 + 2 * random()
        val3 = random()
        return val1, val2, val3
        
        
class Manip2(ManipBase):
    
    name = 'B'
    data_types = ('T (°C)', 'valve')
    
    def _get_values(self):
        val1 = 20 + random()
        val2 = int(random() * 100)
        return val1, val2

In [4]:
data_types = {'A': Manip1.data_types,
              'B': Manip2.data_types}

data_ranges = {'P (Pa)': (0, 1),
               'T (°C)': (20, 30),
               'valve': (0, 100)}

In [5]:
manip1 = Manip1(interval=0.02)
manip1.start()

manip2 = Manip2(interval=0.1)
manip2.start()

In [6]:
colors = {'fig': 'lightgrey',  # set color of figure background around axes
          'B': ('k', 'b')}      # set color of curves of the two channels of sensor B

# Regular numerical plot

In [ ]:
numplot = NumericalGraph(names=('A', 'B'),
                         data_types=data_types,
                         linestyle='.')

numplot.run(q_plot={'A': manip1.queue,
                    'B': manip2.queue},
            dt_graph=0.2)

# Oscilloscope-like graph

In [ ]:
oscillo = OscilloGraph(names=('A', 'B'),
                       data_types=data_types,
                       data_ranges=data_ranges,
                       window_width=10,
                       colors=colors,
                       linestyle='.-')

oscillo.run(q_plot={'A': manip1.queue, 'B': manip2.queue}, dt_graph=0.02)

# Stop dummy sensors

In [ ]:
manip1.stop()
manip2.stop()